# Set Up

In [ ]:
# !pip install -qU transformers==4.48.3 datasets==2.18.0 optimum==1.24.0
# !pip install -qU json-repair==0.29.1

# Libraries Import

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from os.path import join

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
from data_model import getDetailExtractionMessage, getTranslationMessage

# Constants

In [ ]:
BASE_MODEL_ID = "Qwen/Qwen2.5-1.5B-Instruct"
TORCH_TYPE = None

# Model (Base Model)

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_ID,
    device_map="auto",
    torch_dtype = TORCH_TYPE,
    offload_folder="./offload_dir"
)

tokenizer_temp = AutoTokenizer.from_pretrained(BASE_MODEL_ID)

In [ ]:
base_model

# Model Prediction

In [ ]:
def predict(message: str):
    text = tokenizer_temp.apply_chat_template(
        message,
        tokenize=False,
        add_generation_prompt=True
    )

    model_inputs = tokenizer_temp([text], return_tensors="pt").to(base_model.device)

    generated_ids = base_model.generate(
        model_inputs.input_ids,
        max_new_tokens=1024,
        do_sample=False, top_k=None, temperature=None, top_p=None,
    )

    generated_ids = [
        output_ids[len(input_ids):]
        for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer_temp.batch_decode(generated_ids, skip_special_tokens=True)[0]

    return response

In [ ]:
story = """
أنهى مسئولو النادى الأهلى كل تفاصيل عقد أحمد مصطفى زيزو، جناح الزمالك، بحيث ينضم اللاعب لمدة 4 سنوات قادمة بدءا من الموسم الجديد، فى صفقة انتقال حر بعد نهاية عقده مع الزمالك عقب نهاية الموسم الحالى.

ووضع الأهلى زيزو فى الفئة الأولى بحيث يحصل على 25 مليون جنيه سنويا بالإضافة إلى عقد إعلانى مقابل 60 مليون جنيه، بخلاف 80 مليون جنيه أخرى سيحصل عليها دفعة واحدة من خارج النادى عند الإعلان الرسمى عن الصفقة.

وينتهى عقد أحمد سيد زيزو مع الزمالك بنهاية الموسم الجارى، مما يجعل له الحق فى التفاوض مع أى نادٍ وإتمام الاتفاق معه والانضمام له عقب نهاية عقده الحالي خلال يونيو المقبل.

وحصل زيزو على تأشيرة السفر لأمريكا بالفعل للمشاركة مع الأهلى في مونديال الأندية الصيف المقبل.

ويستعد الأهلى للمشاركة فى بطولة كأس العالم للأندية بشكلها الجديد فى الولايات المتحدة الأمريكية، والمقامة بمشاركة 32 ناديًا من أكبر فرق العالم
"""

Before Tuning

In [ ]:
getDetailExtractionMessage(story)

In [ ]:
predict(getDetailExtractionMessage(story))

In [ ]:
getTranslationMessage(story)

In [ ]:
predict(getTranslationMessage(story))

# Loading Adaptor

In [ ]:
# finetuned_model_id = "models"
# base_model.load_adapter(finetuned_model_id)